In [2]:
import pandas as pd
import numpy as np
import math
import sys
sys.path.append("../../src")
from preprocessing import preprocess 

#### Loading main dataset and dataset with site information

To begin with the main dataset is loaded as well as the dataset with the site information

In [4]:
df = pd.read_excel("../data/matrixdata.xls", na_filter = False)

# Removing last row which included the totals
df.drop(df.tail(1).index,inplace=True) 
df.tail()


,RAAD type number,RAAD form,Origin,Contents,Southampton type number,1,2,3,4,5,...,70,71,72,73,74,75,76,77,78,79
263,264,Dolium,Unknown,Unknown,,,,,,,...,,,,,,,,,,
264,265,Dressel 2-4,Unknown,Wine,,,,,,,...,,,,,,,,,,
265,266,Dressel 2-5,Unknown,Wine,,,,,,,...,,,,,,,,,,
266,267,Unknown,Unknown,Unknown,,2,228,,2,,...,151,,,10,4,,1,,4,10
267,268,Augst 30,Unknown (Iberian Peninsula?),,,,,,,,...,,,,,,,,,,


The columns in the site dataframe are renamed not to include spaces, capital letters and parenthesis.

In [3]:
site = pd.read_excel("../data/site.xls")

site = site.rename(columns = {
                       'Assemblage number': 'site_number', 
                       'Site name (modern)': 'site_name_modern',
                       'Site name (ancient)':'site_name_ancient',
                       'Modern country':'modern_country',
                       'Roman Province':'roman_province',
                       'Pleiades':'pleiades',
                       'DARE':'dare',
                       'Vici':'vici',
                       'Lat':'lat',
                       'Long':'long',
                       'Major site type':'major_site_type', 
                       'Minor site type':'minor_site_type',
                       'Start date (AD)':'site_start_date',
                       'End date (AD)':'site_end_date',
                       'Quantification method':'quantification_method', 
                       'Quantification abbreviation':'quantification_abbreviation',
                       'Total assemblage size':'total_assemblage_size',
                       'Reference':'reference'
                       })
#site.head()

In [5]:
raadtypedates = pd.read_excel("../data/RAAD_conflicting-dates_TF.xlsx", usecols=['RAAD type number', 'State Date', 'End Date'])

raadtypedates = raadtypedates.rename(columns={
                                         'RAAD type number' : 'RAAD_type_number', 
                                         'State Date': 'raad_type_start_date',
                                         'End Date': 'raad_type_end_date'
                                         })
raadtypedates.head()

,RAAD_type_number,raad_type_start_date,raad_type_end_date
0,1,NaN,NaN
1,2,1.0,100.0
2,3,-50.0,400.0
3,4,300.0,450.0
4,5,400.0,500.0


The aim is now, that instead of having the matrix format of the dataset it should be restructured in such a way that it includes a row for each site that each RAAD type occurs in.


In [6]:
# Defining the names of the columns we want
colnames=['site_number','RAAD_type_number', 'RAAD_form', 'origin', 'contents', 'southampton_type_number', 'frequency']

# Creating an empty dataframe with these columns
df_loop = pd.DataFrame(columns = colnames)

In [7]:
for row in range(df.shape[0]):
    for i in range(df.columns.get_loc(1), df.columns.get_loc(79)+1):
        if df.iloc[row, i] != '':
            row_info = {
                'RAAD_type_number': df.iloc[row, 0],
                'RAAD_form': df.iloc[row, 1],
                'origin': df.iloc[row, 2],
                'contents': df.iloc[row, 3],
                'southampton_type_number': df.iloc[row, 4], 
                'site_number': df.columns[i],
                'frequency': df.iloc[row, i],
                }
            df_loop = df_loop.append(row_info, ignore_index=True)

df_loop

/tmp/ipykernel_13733/2089385033.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_loop = df_loop.append(row_info, ignore_index=True)
/tmp/ipykernel_13733/2089385033.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_loop = df_loop.append(row_info, ignore_index=True)
/tmp/ipykernel_13733/2089385033.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_loop = df_loop.append(row_info, ignore_index=True)
/tmp/ipykernel_13733/2089385033.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_loop = df_loop.append(row_info, ignore_index=True)
/tmp/ipykernel_13733/2089385033.py:13: FutureWarning: The frame.append method is depreca

,site_number,RAAD_type_number,RAAD_form,origin,contents,southampton_type_number,frequency
0,9,1,Augst 48,EM,Unknown,,1
1,9,2,Augst 49,EM,Unknown,,1
2,9,3,Augst 55/ Agora F65-66,EM,Wine,10,12
3,9,4,Augst 56,EM,Unknown,,2
4,9,5,Augst 57,EM,Unknown,,1
...,...,...,...,...,...,...,...
1464,78,267,Unknown,Unknown,Unknown,,4
1465,79,267,Unknown,Unknown,Unknown,,10
1466,7,268,Augst 30,Unknown (Iberian Peninsula?),,,1
1467,9,268,Augst 30,Unknown (Iberian Peninsula?),,,77


The origin varible is hierachical. We want two new variables; the more specific level(h2) and the less specific(h1).

In [8]:
def get_h1(origin, substring):
    origin = str(origin)

    if substring in origin:
        new = origin.split('-')[0]
        return new
    else:
        return origin

def get_h2(origin, substring):
    if substring in origin:
        new = origin.split('-')[1]
        return new
    else:
        return ''

In [9]:
df_loop['origin_h1'] = [get_h1(row, ' -') for row in df_loop['origin']]

df_loop['origin_h2'] = [get_h2(row, '- ') for row in df_loop['origin']]

Alot of the levels are the same, but accidently has an extra space. These are removed

In [10]:
def clean_column(dataframe, column):
    clean = dataframe[column].str.replace(' ', '')
    clean = clean.str.replace('?', '') 
    clean = clean.str.replace('()', '')
    return clean

In [11]:
df_loop['origin_h1'] = clean_column(df_loop,'origin_h1')
df_loop['origin_h2'] = clean_column(df_loop, 'origin_h2')


/tmp/ipykernel_13733/2804393150.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  clean = clean.str.replace('?', '')
/tmp/ipykernel_13733/2804393150.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  clean = clean.str.replace('()', '')


In [11]:
df_loop['origin_h1'].unique()

array(['EM', 'Gaul', 'Germania', 'Italy', 'NorthAfrica', 'Raetia',
       'IberianPeninsula', 'Unknown', 'Unknown(IberianPeninsula)'],
      dtype=object)

Now that the data has been restructed, the site information is merged together with this dataset. This way all rows include some site information as well

In [12]:
df_merged = pd.merge(df_loop, raadtypedates, how = 'inner', on = 'RAAD_type_number')
df_merged = pd.merge(df_merged, site, how = 'inner', on = 'site_number')
df_merged.columns
df_merged.head()

,site_number,RAAD_type_number,RAAD_form,origin,contents,southampton_type_number,frequency,origin_h1,origin_h2,raad_type_start_date,...,lat,long,major_site_type,minor_site_type,site_start_date,site_end_date,quantification_method,quantification_abbreviation,total_assemblage_size,reference
0,9,1,Augst 48,EM,Unknown,,1,EM,,NaN,...,47.533512,7.71628,Settlement/ Military,Colonia/ Limitanei,-10,450,Total sherds,Total,5633,"Martin-Kilcher, S. (1987). Die rˆmischen Ampho..."
1,9,2,Augst 49,EM,Unknown,,1,EM,,1.0,...,47.533512,7.71628,Settlement/ Military,Colonia/ Limitanei,-10,450,Total sherds,Total,5633,"Martin-Kilcher, S. (1987). Die rˆmischen Ampho..."
2,9,3,Augst 55/ Agora F65-66,EM,Wine,10,12,EM,,-50.0,...,47.533512,7.71628,Settlement/ Military,Colonia/ Limitanei,-10,450,Total sherds,Total,5633,"Martin-Kilcher, S. (1987). Die rˆmischen Ampho..."
3,9,4,Augst 56,EM,Unknown,,2,EM,,300.0,...,47.533512,7.71628,Settlement/ Military,Colonia/ Limitanei,-10,450,Total sherds,Total,5633,"Martin-Kilcher, S. (1987). Die rˆmischen Ampho..."
4,9,5,Augst 57,EM,Unknown,,1,EM,,400.0,...,47.533512,7.71628,Settlement/ Military,Colonia/ Limitanei,-10,450,Total sherds,Total,5633,"Martin-Kilcher, S. (1987). Die rˆmischen Ampho..."


In [13]:
df_merged.columns

cols_for_preprocessing = [
    'RAAD_form', 'origin', 'contents',
    'origin_h1', 'origin_h2', 'site_name_modern',
    'site_name_ancient', 'modern_country', 'roman_province',
    'major_site_type', 'minor_site_type', 'quantification_method',
    'quantification_abbreviation', 'reference'
    ]

for i in cols_for_preprocessing:
    df_merged[i] = preprocess(df_merged[i])

In [14]:
df_merged.to_csv("../data/RAAD_data_restructured.csv")